In [1]:
import sys
import os
from PyQt5.QtWidgets import QApplication, QWidget, QLCDNumber
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QFont
import ctypes
import configparser

class OverlayTimer(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.config = configparser.ConfigParser()
        self.ini_path = 'timer_config.ini'
        self.load_config()

        # 위젯 설정
        self.setWindowTitle('PoE Timer')
        self.setWindowFlags(Qt.WindowStaysOnTopHint | Qt.FramelessWindowHint | Qt.Tool)
        self.setAttribute(Qt.WA_TranslucentBackground)

        # LCD 숫자 디스플레이 설정
        self.lcd = QLCDNumber(self)
        self.lcd.setDigitCount(8)
        self.lcd.setSegmentStyle(QLCDNumber.Flat)
        self.lcd.setStyleSheet("background-color: rgba(0, 0, 0, 100); color: rgb(255, 0, 0);")

        # 타이머 설정
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.showTime)
        self.seconds = 0
        self.is_running = False

        # 화면 크기 체크 타이머
        self.screen_check_timer = QTimer(self)
        self.screen_check_timer.timeout.connect(self.check_screen_size)
        self.screen_check_timer.start(1000)  # 1초마다 화면 크기 체크

        # 키 입력 체크 타이머
        self.key_check_timer = QTimer(self)
        self.key_check_timer.timeout.connect(self.check_keys)
        self.key_check_timer.start(100)  # 0.1초마다 키 입력 체크

        self.update_size_and_position()
        self.showTime()

    def load_config(self):
        if os.path.exists(self.ini_path):
            self.config.read(self.ini_path)
        else:
            self.config['Position'] = {'x': '0.9', 'y': '0.05'}
            self.config['Size'] = {'width': '0.05', 'height': '0.025'}
            self.save_config()

    def save_config(self):
        with open(self.ini_path, 'w') as configfile:
            self.config.write(configfile)

    def update_size_and_position(self):
        user32 = ctypes.windll.user32
        self.screen_width = user32.GetSystemMetrics(0)
        self.screen_height = user32.GetSystemMetrics(1)

        width_ratio = self.config.getfloat('Size', 'width', fallback=0.05)
        height_ratio = self.config.getfloat('Size', 'height', fallback=0.025)
        x_ratio = self.config.getfloat('Position', 'x', fallback=0.9)
        y_ratio = self.config.getfloat('Position', 'y', fallback=0.05)

        self.timer_width = int(self.screen_width * width_ratio)
        self.timer_height = int(self.screen_height * height_ratio)
        pos_x = int(self.screen_width * x_ratio)
        pos_y = int(self.screen_height * y_ratio)

        self.setGeometry(pos_x, pos_y, self.timer_width, self.timer_height)
        self.lcd.resize(self.timer_width, self.timer_height)

    def check_screen_size(self):
        user32 = ctypes.windll.user32
        new_width = user32.GetSystemMetrics(0)
        new_height = user32.GetSystemMetrics(1)
        if new_width != self.screen_width or new_height != self.screen_height:
            self.update_size_and_position()

    def showTime(self):
        if self.is_running:
            self.seconds += 1
        m, s = divmod(self.seconds, 60)
        h, m = divmod(m, 60)
        time = f"{h:02d}:{m:02d}:{s:02d}"
        self.lcd.display(time)

    def start_timer(self):
        self.is_running = True
        self.timer.start(1000)

    def toggle_timer(self):
        if self.is_running:
            self.timer.stop()
        else:
            self.timer.start(1000)
        self.is_running = not self.is_running

    def reset_timer(self):
        self.seconds = 0
        self.showTime()

    def check_keys(self):
        modifiers = QApplication.keyboardModifiers()
        if modifiers == Qt.NoModifier:
            if QApplication.queryKeyboardModifiers() & Qt.KeypadModifier:
                if ctypes.windll.user32.GetAsyncKeyState(0x71) & 0x8000:  # F2
                    self.reset_timer()
                elif ctypes.windll.user32.GetAsyncKeyState(0x72) & 0x8000:  # F3
                    self.start_timer()
                elif ctypes.windll.user32.GetAsyncKeyState(0x73) & 0x8000:  # F4
                    self.toggle_timer()
                elif ctypes.windll.user32.GetAsyncKeyState(0x76) & 0x8000:  # F7
                    self.close()

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.dragPosition = event.globalPos() - self.frameGeometry().topLeft()
            event.accept()

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            self.move(event.globalPos() - self.dragPosition)
            event.accept()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.save_position()

    def save_position(self):
        x = self.x() / self.screen_width
        y = self.y() / self.screen_height
        self.config['Position'] = {'x': str(x), 'y': str(y)}
        self.save_config()

    def closeEvent(self, event):
        self.save_position()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = OverlayTimer()
    ex.show()
    sys.exit(app.exec_())


KeyboardInterrupt: 